In [ ]:
import mingus 
import fluidsynth as fs
import time
from midi2audio import FluidSynth
import mingus.midi
from mingus.midi.midi_file_out import write_Track
from mingus.midi import fluidsynth
from mingus.containers import Note
import mingus.core.notes as notes
from mingus.containers import Track

In [ ]:
t = Track()
for note in ["G","B","C","D","E","F"]:
    t.add_notes(note)

# t = Track().from_chords([["Cm/M7|FM"]], 1)

In [ ]:
sf2 = "../sounds/198-Stratocaster VS.sf2"
fluidsynth.init(sf2);
fluidsynth.play_Track(t);
write_Track("test.midi", t);
fs = FluidSynth(sf2)
fs.midi_to_audio("test.midi", 'input.wav')

In [ ]:
# https://stackoverflow.com/questions/27028157/writing-a-program-converts-guitar-tabs-to-notes
NOTES = {"C" : 0, "C#" : 1,  "D": 2, "D#" : 3, "E": 4, "F": 5, 
   "F#" : 6, "G":  7, "G#" : 8, "A": 9, "A#" : 10, "B": 11}
TUNING = {"E" : 5, "A" : 4, "D" : 3, "G" : 2, "B" : 1, "E" : 0}

NAMES = dict([(v, k) for (k, v) in NOTES.items()])
TUNING = dict([(v, k) for (k, v) in TUNING.items()])

def GetNote(baseNote, fretNum):
    if baseNote == -1:
        return None
    fretNoteNum = (baseNote + fretNum) % 12
    return NAMES[fretNoteNum]

In [ ]:
tab = """
––––––––––––––––3––3–3––3–––––––––––––––––3––3–3––3–––
––––––––––––––––3––3–3––3–––––––––––––––––3––3–3––3–––
––––––––––––––––0––0–0––0––––––––0––––––––0––0–0––0–––
–––––––––0–2––––2––2–2––2––––––2–––2–0––––0––0–0––0–––
–––––02–––––––––––––––––––––––––––––––––––––––––––––––
–––3––––––––––––––––––––––––––––––––––––––––––––––––––
"""

In [ ]:
import numpy as np
np_tab = np.array([list(string) for string in tab.split('\n')[1:7]])
np_tab = np.where(np_tab == '–', '-1', np_tab).astype(int)

chords = []
for i in range(np_tab.shape[1]):
    chord = [GetNote(j,i) for j in np_tab[:,i] if GetNote(j,i) is not None]
    chords.append(chord)
    
t = Track().from_chords(chords,1)

In [ ]:
from scipy.interpolate import interp1d
from scipy.io import wavfile

In [ ]:
def apply_transfer(signal, transfer, interpolation='linear'):
    constant = np.linspace(-1, 1, len(transfer))
    interpolator = interp1d(constant, transfer, interpolation)
    return interpolator(signal)

# hard limiting
def limiter(x, treshold=0.8):
    transfer_len = 1000
    transfer = np.concatenate([ np.repeat(-1, int(((1-treshold)/2)*transfer_len)),
                                np.linspace(-1, 1, int(treshold*transfer_len)),
                                np.repeat(1, int(((1-treshold)/2)*transfer_len)) ])
    return apply_transfer(x, transfer)

# smooth compression: if factor is small, its near linear, the bigger it is the
# stronger the compression
def arctan_compressor(x, factor=2):
    constant = np.linspace(-1, 1, 1000)
    transfer = np.arctan(factor * constant)
    transfer /= np.abs(transfer).max()
    return apply_transfer(x, transfer)

In [ ]:
sr, x = wavfile.read("input.wav")
x = x / np.abs(x).max() # x scale between -1 and 1

x2 = limiter(x)
x2 = np.int16(x2 * 32767)
wavfile.write("output_limit.wav", sr, x2)

x3 = arctan_compressor(x)
x3 = np.int16(x3 * 32767)
wavfile.write("output_comp.wav", sr, x3)

In [ ]:
wavfile.write("white_noise.wav", 25000, np.random.rand(700000,2))

In [ ]:
wave = list(0.5*np.sin(0.05*np.linspace(0,500000,1000000)))
r = wave * np.maximum(np.sin(0.00005*np.linspace(0,500000,1000000)),0)
l = wave * np.maximum(-np.sin(0.00005*np.linspace(0,500000,1000000)),0)
wavfile.write("white_noise.wav", 48000, np.array([r,l]).T)